## 1.0 Load libraries and read data

In [8]:
import ee
import os
import geemap
import json
import geopandas as gpd
import pprint as pp
from shapely.geometry import shape

ee.Authenticate()
ee.Initialize()

In [9]:
project_dir = 'projects/alpod-412314/assets/' 
region = 'TUK_MRD'

region_path = f'{region}_weekly'
folder = project_dir + region_path
#pp.pp(ee.data.listAssets(project_dir))
#pp.pp(ee.data.listAssets(project_dir + 'region_weekly/'))
#pp.pp(ee.data.listAssets(f'{project_dir}Lake_extractions/'))

# Read bufferd lake polygons
region_buffered_lakes = 'TUK_MRD_AND' #!!! TUK_MRD is called TUK_MRD_AND for the lake regions
lakes_path = f'{project_dir}Lake_extractions/{region_buffered_lakes}_extraction'
lake_polygons = ee.FeatureCollection(lakes_path)

# Read weekly mosaics
weekly_mosaics = ee.data.listAssets(project_dir + 'region_weekly')
weekly_mosaics = weekly_mosaics['assets']

## 2.0 Select weekly mosaics for given timeperiods and ROI

In [10]:
target_years = list(range(2016, 2024))
target_years = [str(year) for year in target_years]

# June weeks !!!
#target_weeks = list(range(22, 27))
# August weeks !!!
target_weeks = list(range(31, 36))

target_weeks = [str(week) for week in target_weeks]

max_observations = len(target_years) * len(target_weeks)

target_imgs = []
target_imgs_footprints = []

for mosaic in weekly_mosaics:
    img_id = mosaic['id']
    region_match = region.split('_')[-1]
    temp_region = img_id.split('_')[-3]
    temp_region = temp_region.split('/')[-1]
    temp_year = img_id.split('_')[-2]
    temp_week = img_id.split('_')[-1]

    if temp_week in target_weeks and temp_year in target_years and temp_region == region_match:
        #print(img_id)
        image = ee.Image(img_id)
        image_info = image.getInfo()
        #pp.pp(image_info)
        image_coords = image_info['properties']['system:footprint']['coordinates']
        image_polygon_ee = ee.Geometry.Polygon(image_coords)

        target_imgs.append(image)
        target_imgs_footprints.append(image_polygon_ee)

"""
A bit hacky, 
but I combine all the image footprints.
This generates a bounding box for the entire ROI
"""

polygon1 = target_imgs_footprints[0]
polygon2 = target_imgs_footprints[1]
combined_footprint = polygon1.union(polygon2)
for i in range(2, len(target_imgs_footprints)):
    p = target_imgs_footprints[i]
    combined_footprint = combined_footprint.union(p)

"""
Write the combined footprint as a bounding box shapefile.
Might be usefull later. 
"""
combined_footprint_info = combined_footprint.getInfo()
geojson_footprint = json.dumps(combined_footprint_info)
geometry = shape(json.loads(geojson_footprint))
roi_gdf = gpd.GeoDataFrame([{'geometry': geometry, 'roi': f'{region}'}], crs="EPSG:4326")
# os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')
# roi_gdf.to_file(f'./data/ew_rois/{region}_bbox.shp')


## 3.0 Make a classified image (maybe refactor code?)

In [11]:
"""
From the weekly mosaic AND the original buffered lake polygons, produce a new image.
The new image has one band with integers for all conditions.
0 = Never observed, outside of ROI OR not a prior buffered lake
1 = A valid observation of land on a buffered lake polygon
2 = A valid observation of water on a buffered lake polygon
3 = An invalid observatoin where could or ice cover obscured a lake polygon

!!! There's definitely a cleaner way to do this, I might refactor code later.
"""

classified_images = []

for img in target_imgs:

    # Mask of valid observations over buffered lake polygons no clouds or ice
    # !!! Probably not necessary
    obs_mask = img.unmask(0)
    obs_mask = obs_mask.rename('valid_observation')
    obs_mask = obs_mask.clip(combined_footprint)
    #print(obs_mask.getInfo())
    
    # Mask of valid water observations
    img_wtr = ee.Image.constant(0)
    img_wtr = img_wtr.where(img.select('water_occurance').eq(1), 2)
    img_wtr = img_wtr.rename('wtr_occurance')
    img_wtr = img_wtr.clip(combined_footprint)
    
    # Mask of valid land observations
    img_land = ee.Image.constant(0)
    img_land = img_land.where(img.select('water_occurance').eq(0), 1)
    img_land = img_land.rename('land_occurance')
    img_land = img_land.clip(combined_footprint)
    
    # Mask of prior lake polygons
    lakes_binary = lake_polygons.reduceToImage(
        properties=['n_lakes'],
        reducer=ee.Reducer.anyNonZero()
    ).neq(0)
    lakes_binary = lakes_binary.rename('buffered_lake')
    lakes_binary = lakes_binary.clip(combined_footprint)

    expr = """
    (wtr_observed == 2) ? 2 :
    (land_observed == 1) ? 1 :
    ((buffered_lake == 1) && (land_observed == 0) && (wtr_observed != 2)) ? 3 :
    0
    """
    classified = ee.Image.constant(0)
    classified = lakes_binary.expression(
        expr,
        {
            'wtr_observed': img_wtr.select('wtr_occurance'),
            'land_observed': img_land.select('land_occurance'),
            'buffered_lake': lakes_binary.select('buffered_lake'),
        }
    )
    
    classified = classified.rename('class')
    #print(classified.getInfo())

    classified_images.append(classified)

## 4.0 Calculate the proportion of water occurance in valid observations (maybe refactor later?)

In [12]:
"""
Create a collection...
Then, calculate the total valid water observations 
AND invalid (ice/cloud) observations for each pixel
"""
classified_collection = ee.ImageCollection(classified_images)

def mask_and_set_to_one(image, class_value):
    return image.updateMask(image.eq(class_value)).multiply(0).add(1)

# Create images for each class
land_images = classified_collection.map(lambda img: mask_and_set_to_one(img.select('class'), 1))
wtr_images = classified_collection.map(lambda img: mask_and_set_to_one(img.select('class'), 2))
inval_images = classified_collection.map(lambda img: mask_and_set_to_one(img.select('class'), 3))

# Sum the images
wtr_sum_img = wtr_images.sum()
#land_sum_img = land_images.sum()
inval_observations_sum_img = inval_images.sum()

"""
Make a constant image for the max possible observations (i.e. all weeks cloud/ice free)
For each pixel...
Valid water occurance fraction = [total water observation / (max possible observations - invalid observations)]
"""
# Calc % water for observed images across each pixel
total_obs = len(target_imgs)
print(f'Highest possible total weeks {max_observations}, but observed total weeks {total_obs}')

max_obs_image = ee.Image.constant(total_obs).clip(combined_footprint)
wtr_occurance_frac = wtr_sum_img.divide(max_obs_image.subtract(inval_observations_sum_img))

# Example: Print the result
pp.pp(wtr_occurance_frac.getInfo())

Highest possible total weeks 40, but observed total weeks 38
{'type': 'Image',
 'bands': [{'id': 'class',
            'data_type': {'type': 'PixelType', 'precision': 'float'},
            'dimensions': [10, 5],
            'origin': [-138, 66],
            'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:footprint': {'type': 'Polygon',
                                     'coordinates': [[[-135.55053740646167,
                                                       67.4959965388688],
                                                      [-135.15298776666535,
                                                       67.4968094504536],
                                                      [-134.82164782646063,
                                                       67.49619972858332],
                                                      [-134.68911123695298,
                                                       67.49652493345724],
              

In [ ]:
wtr_frac_viz = {
    'bands': ['class'],
    'min': 0,
    'max': 1,
    'palette': ['brown', 'green', 'blue']
}


Map = geemap.Map(center=(65, -135), zoom=4)
Map.addLayer(wtr_occurance_frac, wtr_frac_viz, 'water frac')
Map

In [14]:
task = ee.batch.Export.image.toDrive(
    image=wtr_occurance_frac,
    description=f'{region}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
    folder='sentinel2_exports',
    fileNamePrefix=f'{region}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
    scale=10,
    crs='EPSG:4326',
    maxPixels=1e13
)

task.start()

In [ ]:
# band = wtr_occurance_frac.select('class')

# coordinates = [
#     [-146.42929053, 66.34721564],
#     [-145.73500565, 66.32739946],
#     [-145.74722181, 66.15123003],
#     [-146.51124058, 66.15822638]
# ]

# # Create the polygon geometry
# t = ee.Geometry.Polygon([coordinates])

# histogram = band.reduceRegion(
#     reducer=ee.Reducer.fixedHistogram(min=0, max=1, steps=20),
#     geometry=combined_footprint,
#     scale=10,
#     maxPixels=1e13,
# )
# result = histogram.getInfo()

# pp.pp(result)import matplotlib.pyplot as plt

# class_values, frequencies = zip(*result['class'])

# # Create a bar plot
# plt.figure(figsize=(12, 6))
# plt.bar(class_values, frequencies, width=0.03, align='center')
# plt.xticks(class_values, rotation=45)  # Rotate x-ticks for better visibility
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()

In [ ]:
# obs_viz = {
#     'bands': ['observed'],
#     'min': 0,
#     'max': 1,
#     'palette': ['red', 'green']
# }
# lakes_viz = {
#     'bands': ['buffered_lake'],
#     'min': 0,
#     'max': 1, 
#     'palette': ['white', 'blue']
# }
# qk_viz = {
#     'bands': ['wtr_occurance'],
#     'min': 0,
#     'max': 2, 
#     'palette': ['white', 'blue']
# }
# qk_viz2 = {
#     'bands': ['land_occurance'],
#     'min': 0,
#     'max': 1, 
#     'palette': ['white', 'brown']
# }

# conditional_palette = ['grey', 'brown', 'blue', 'pink']
# conditional_viz = {
#     'band': ['class'],
#     'min': 0,
#     'max': 3,
#     'palette': conditional_palette,
# }

# conditional_test = ee.Image(conditional_images[22])

# # Create a map to display the results
# Map = geemap.Map(center=(65, -135), zoom=4)
# #Map.addLayer(obs_mask, obs_viz, 'Observation Mask')
# Map.addLayer(conditional_test, conditional_viz, 'Conditional Test')
# #Map.addLayer(lakes_binary, lakes_viz, 'Buffered Lakes')
# #Map.addLayer(img_wtr, qk_viz, 'water')
# #Map.addLayer(img_land, qk_viz2, 'land?')

# Map